<a href="https://colab.research.google.com/github/adolfolh/casuality-classification/blob/main/modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **`3.` MODELLING** 📈

In [ ]:
# Run to download data
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/X_train.csv
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/y_train.csv
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/X_test.csv
!wget https://github.com/adolfolh/casuality-classification/raw/main/data/y_test.csv

In [5]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import metrics

In [6]:
# Import data
X_train = pd.read_csv(r'X_train.csv', index_col='Unnamed: 0')
y_train = pd.read_csv(r'y_train.csv', index_col='Unnamed: 0')
X_test = pd.read_csv(r'X_test.csv', index_col='Unnamed: 0')
y_test = pd.read_csv(r'y_test.csv', index_col='Unnamed: 0')

In [7]:
print(f"Shape of training features: {X_train.shape}")
print(f"Shape of training labels: {y_train.shape}")
print(f"Shape of testing features: {X_test.shape}")
print(f"Shape of testing labels: {X_test.shape}")

Shape of training features: (78634, 11)
Shape of training labels: (78634, 1)
Shape of testing features: (19659, 11)
Shape of testing labels: (19659, 11)


In [8]:
# Create Decision Tree classifer object
clf = tree.DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train, y_train)

In [9]:
#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [10]:
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.7615341573833867
